In [29]:
import os 

os.chdir('c:/Users/Domen/IronHack/01_projects/IH_final_project_data-jobs')
print(os.getcwd())
# Importing necessary libraries

c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs


In [30]:
print(os.getcwd())

c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs


In [31]:
import winsound
import pygame
from bs4 import BeautifulSoup, NavigableString
import requests
from time import sleep
from random import randint
import math
from tqdm import tqdm
import pickle
import pandas as pd
from datetime import date
from datetime import datetime
import json

import pymysql                        # for getting data from a SQL database
from sqlalchemy import create_engine, text  # for establishing the connection and authentication
import zlib


# Importing necessary libraries

In [32]:

from src.myModules import my_NLP as myNLP 
from src.myModules import scraper_ds as mybib
from src.myModules import prep as myPrep

%load_ext autoreload
%autoreload 2
# Importing necessary libraries

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
print(os.getcwd())
with open("config/config.json", 'r') as f:
    config = json.load(f)
    
temp_data_path = config["temp_data_path"]
webscrap_data_path = config["webscrap_data_path"]    

print(temp_data_path)
print(webscrap_data_path)


soup_file =  'soup_dict'
keyword_file = 'keyword_dict'
dataframe_output = 'transformed_dataframe'


c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs
data/temp_data/
data/webscrap_data/


### load scraped raw data

In [49]:

with open(f"{webscrap_data_path}{soup_file}.pkl", "rb") as f:
    scraper_dict = pickle.load(f)
    
with open(f"{webscrap_data_path}{keyword_file}.pkl", "rb") as f:
    keyword_dict = pickle.load(f)

In [35]:
print(len(scraper_dict))
print(len(keyword_dict))


22079
21879


In [48]:

with open(f"{webscrap_data_path}soup_dictV2.pkl", "rb") as f:
    new_scraper_dict = pickle.load(f)
    
with open(f"{webscrap_data_path}keyword_dictV2.pkl", "rb") as f:
    new_keyword_dict = pickle.load(f)

In [37]:
print(len(new_scraper_dict))
print(len(new_keyword_dict))

14306
14306


In [50]:
keyword_dict.update(new_keyword_dict)
scraper_dict.update(new_scraper_dict)

In [39]:
print(len(scraper_dict))
print(len(keyword_dict))

29302
29265


### transform scraped raw data into df 

In [40]:
df = myPrep.transform_scrap_data_to_df(scraper_dict)

100%|██████████| 29302/29302 [19:28<00:00, 25.07it/s]  


In [42]:
df["title"].isna().sum()

443

### data cleaning - empty rows
Trying to fill none values doing a second request

NameError: name 'empty_rows' is not defined

In [44]:
cleaned_df = myPrep.null_value_cleaner(df, 4) # increases each round the sleep time for scraping process

100%|██████████| 443/443 [00:12<00:00, 36.75it/s]


Round0: 26 empty values left


100%|██████████| 26/26 [00:00<00:00, 35.33it/s]


Round1: 0 empty values left
Finished in round <built-in function id>


### Add keywords

In [51]:
new_keyword_dict

{'3719620327': ['Data Analyst',
  'Data Engineer',
  'Data Visualization Specialist',
  'Data Consultant',
  'Data Manager',
  'Data Quality Analyst',
  'data_analyst',
  'data_engineer',
  'product_analyst',
  'data_scientist'],
 '3724233663': ['Data Analyst',
  'Data Engineer',
  'Data Architect',
  'Data Visualization Specialist',
  'Data Consultant',
  'Data Manager',
  'Data Quality Analyst',
  'data_analyst',
  'data_scientist',
  'data_analyst',
  'data_engineer',
  'data_scientist'],
 '3722456887': ['Data Analyst',
  'Data Engineer',
  'Data Engineer',
  'Data Architect',
  'Data Architect',
  'Data Visualization Specialist',
  'Data Consultant',
  'Data Manager',
  'Data Quality Analyst',
  'data_analyst',
  'data_engineer',
  'data_engineer',
  'data_analyst',
  'data_engineer',
  'data_scientist'],
 '3713133845': ['Data Analyst',
  'Data Architect',
  'Data Visualization Specialist',
  'Data Consultant',
  'Data Manager',
  'data_analyst',
  'data_engineer',
  'data_analyst'

In [52]:
small_dict = {}
for key in keyword_dict:
    small_dict[key] = keyword_dict[key][0]

In [53]:
cleaned_df['keyword'] = cleaned_df['id'].map(small_dict)

### Add language 

In [54]:

cleaned_df = myNLP.add_language_values_to_df(cleaned_df, search_column = 'job_description') # add language values 


Processing: 100%|██████████| 29302/29302 [05:08<00:00, 94.97it/s] 


### Save data

In [55]:
myPrep.save_cleaned_df(cleaned_df, dataframe_output)